# Iceberg Workshop: Getting Started

## Setup Catalog

In [1]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog("default")

## Example Data

| name    | id | date       |
|---------|----|------------|
| Alice   |  1 | 2018-04-02 |
| Bob     |  2 | 2020-09-07 |
| Charlie |  3 | 2022-07-01 |

In [2]:
import pyarrow as pa

example_schema = pa.schema(
    [
        ("name", pa.string()),
        ("id", pa.int32()),
        ("date", pa.date32()),
    ]
)

example_data = pa.Table.from_pylist([
    {"name": "Alice", "id": 1, "date": 17623},  # 2018-05-15
    {"name": "Bob", "id": 2, "date": 18512},    # 2020-11-23
    {"name": "Charlie", "id": 3, "date": 19174} # 2022-07-07
], schema=example_schema)

## Create an Iceberg table

First, we'll create a namespace `demo_ns` to organize the Iceberg table, then define and create the table using the specified schema.

In [3]:
catalog.create_namespace_if_not_exists("demo_ns")

In [4]:
table = catalog.create_table("demo_ns.demo_table_1", schema=example_schema)
table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date
),
partition by: [],
sort order: [],
snapshot: null

## What happens behind table creation?

A metadata file has been created and registered as the latest metadata of table `demo_ns.demo_table_1`. Let's login to Minio Bucket and see the file:

- Minio Url: http://localhost:9001/
- username: admin
- password: password

The table is created at [s3://warehouse/demo_ns/demo_table_1](http://localhost:9001/browser/warehouse/demo_ns%2Fdemo_table_1%2F): 

![](./imgs/simple_table_create.png)

# Add data to the table

It will create a new snapshot on the table

In [5]:
table.overwrite(example_data)
table

/opt/conda/lib/python3.11/site-packages/pyiceberg/table/__init__.py:686: UserWarning: Delete operation did not match any records
  warnings.warn("Delete operation did not match any records")


demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=1254590870239251066, schema_id=0

## Read the table

We can see example data has been added to the table

In [6]:
table.scan().to_pandas()

,name,id,date
0,Alice,1,2018-04-02
1,Bob,2,2020-09-07
2,Charlie,3,2022-07-01


## What happens when adding data?

The data has been written into a parquet file and a new snapshot has been created.

Let's check the table location again: [s3://warehouse/demo_ns/demo_table_1](http://localhost:9001/browser/warehouse/demo_ns%2Fdemo_table_1%2F)

We can see the table now have both `metadata` and `data`
![](./imgs/simple_table_create_append_data.png)

In the `metadata`, we can see some new files are generated
![](./imgs/simple_table_create_append_data_new_metadata.png)

In the `data`, we can see a new parquet file that contains the inserted data
![](./imgs/simple_table_create_append_data_new_data.png)


## Table Evolution: Make table partitioned

The table we just created is unpartitioned, but it's common practice to partition a table based on specific column(s). No worries—we can easily partition it!

Iceberg allows you to update the partitioning strategy without having to recreate the table or migrate any data.

In [7]:
from pyiceberg.transforms import YearTransform

with table.update_spec() as update:
   update.add_field("date", YearTransform())

table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date
),
partition by: [date_year],
sort order: [],
snapshot: Operation.APPEND: id=7948635370284362920, schema_id=0

## Add more data to partitioned table

New data will be written in to different partitions based on the partition strategy. In this demo, there will be 3 partitions

- date_year=2018
- date_year=2020
- date_year=2022

TODO: Add a preview of current file structure

In [8]:
example_data_2 = pa.Table.from_pylist([
    {"name": "David", "id": 4, "date": 17623},  # 2018-05-15
    {"name": "John", "id": 5, "date": 18512},    # 2020-11-23
    {"name": "Jonas", "id": 6, "date": 19174} # 2022-07-07
], schema=example_schema)

table.append(example_data_2)

In [9]:
table.scan().to_pandas()

,name,id,date
0,David,4,2018-04-02
1,John,5,2020-09-07
2,Jonas,6,2022-07-01
3,Alice,1,2018-04-02
4,Bob,2,2020-09-07
5,Charlie,3,2022-07-01


# Table Evolution: Add new column to the table

Iceberg allows user to add new column(s) without re-create or re-write the table

In [10]:
from pyiceberg.types import StringType, DecimalType

with table.update_schema() as update:
   # Add new columns
   update.add_column("comments", StringType())
   update.add_column("salary", DecimalType(9, 3))

In [11]:
table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date,
  4: comments: optional string,
  5: salary: optional decimal(9, 3)
),
partition by: [date_year],
sort order: [],
snapshot: Operation.APPEND: id=6696808655765377924, parent_id=4688774704661413985, schema_id=0

In [12]:
table.scan().to_pandas()

,name,id,date,comments,salary
0,David,4,2018-04-02,None,None
1,John,5,2020-09-07,None,None
2,Jonas,6,2022-07-01,None,None
3,Alice,1,2018-04-02,None,None
4,Bob,2,2020-09-07,None,None
5,Charlie,3,2022-07-01,None,None


# NYC Taxi Dataset
Now let's use New York City Taxi & Limousine Commission's Trip Record Data to show more features.

In [7]:
import pyarrow.parquet as pq

taxis_data_jan = pq.read_table('/home/jovyan/data/yellow_tripdata_2024-01.parquet')
taxis_data_feb = pq.read_table('/home/jovyan/data/yellow_tripdata_2024-02.parquet')
taxis_data_march = pq.read_table('/home/jovyan/data/yellow_tripdata_2024-03.parquet')
taxis_data_jan.schema

VendorID: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
RatecodeID: int64
store_and_fwd_flag: large_string
PULocationID: int32
DOLocationID: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
Airport_fee: double

# Create an Iceberg Table with that Schema

Let's say we want to partition by the pickup time.

In [9]:
nyc_taxis_tbl = catalog.create_table("demo_ns.nyc_taxis", schema=taxis_data_jan.schema)

## Table Evolution: Make table partitioned

The table we just created is unpartitioned. In this example, we want to take a further step to partition the table. We will partition the table by the `year` value of`tpep_pickup_datatime` column.

In [11]:
from pyiceberg.transforms import YearTransform

with nyc_taxis_tbl.update_spec() as update_spec:
    update_spec.add_field("tpep_pickup_datetime", YearTransform())

nyc_taxis_tbl

nyc_taxis(
  1: VendorID: optional int,
  2: tpep_pickup_datetime: optional timestamp,
  3: tpep_dropoff_datetime: optional timestamp,
  4: passenger_count: optional long,
  5: trip_distance: optional double,
  6: RatecodeID: optional long,
  7: store_and_fwd_flag: optional string,
  8: PULocationID: optional int,
  9: DOLocationID: optional int,
  10: payment_type: optional long,
  11: fare_amount: optional double,
  12: extra: optional double,
  13: mta_tax: optional double,
  14: tip_amount: optional double,
  15: tolls_amount: optional double,
  16: improvement_surcharge: optional double,
  17: total_amount: optional double,
  18: congestion_surcharge: optional double,
  19: Airport_fee: optional double
),
partition by: [tpep_pickup_datetime_year],
sort order: [],
snapshot: null

In [12]:
nyc_taxis_tbl.append(taxis_data_jan)

In [20]:
nyc_taxis_tbl.scan().to_pandas().size

113470850

# Partitioned Data

If we go to the [`data` folder](http://localhost:9001/browser/warehouse/demo_ns%2Fnyc_taxis%2Fdata%2F) of table `nyc_taxis`:

![](./imgs/nyc_year_partition.png)

We can see that inserted data partitioned by year.

## Table Evolution: Change to partition by month

I changed my mind and now I want to partition the table by the "month" of `tpep_pickup_datetime`. No worries—we can easily partition it!

Iceberg allows you to update the partitioning strategy without recreating the table or re-writing any data.

In [16]:
from pyiceberg.transforms import MonthTransform

with nyc_taxis_tbl.update_spec() as update_spec:
    update_spec.remove_field("tpep_pickup_datetime_year")
    update_spec.add_field("tpep_pickup_datetime", MonthTransform())

Now let's append another month of data to the table

In [18]:
nyc_taxis_tbl.append(taxis_data_feb)

If we go to the the [`data` folder](http://localhost:9001/browser/warehouse/demo_ns%2Fnyc_taxis%2Fdata%2F) of table `nyc_taxis` again, we will find the new data is partitioned by the month value

![](./imgs/nyc_month_partition.png)

The previous year partitions' folders are still there because data inserted before partition spec change will remain in their original partition.

In [22]:
nyc_taxis_tbl.specs()

{0: PartitionSpec(spec_id=0),
 1: PartitionSpec(PartitionField(source_id=2, field_id=1000, transform=YearTransform(), name='tpep_pickup_datetime_year'), spec_id=1),
 2: PartitionSpec(PartitionField(source_id=2, field_id=1001, transform=MonthTransform(), name='tpep_pickup_datetime_month'), spec_id=2)}

# Table Evolution: Change Table Schema
Iceberg supports schema evolution without rewriting any data. For example, we can rename `VendorId` to `ID`.



In [28]:
# Before rename
nyc_taxis_tbl.scan(limit=3).to_pandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2009-01-01 23:58:40,2009-01-02 00:01:40,1,0.46,1,N,137,264,2,4.4,3.50,0.5,0.0,0.00,1.0,9.40,0.0,0.0
1,2,2009-01-01 23:30:39,2009-01-02 00:01:39,1,10.99,1,N,237,264,2,45.0,3.50,0.5,0.0,0.00,1.0,50.00,0.0,0.0
2,2,2009-01-01 00:24:09,2009-01-01 01:13:00,2,10.88,1,N,138,264,2,50.6,9.25,0.5,0.0,6.94,1.0,68.29,0.0,0.0


In [30]:
with nyc_taxis_tbl.update_schema() as update:
    update.rename_column("VendorID", "ID")

In [31]:
# After rename
nyc_taxis_tbl.scan(limit=3).to_pandas()

,ID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-31 23:59:53,2024-02-01 00:18:35,1,6.95,1,N,249,166,1,30.3,1.0,0.5,7.06,0.0,1.0,42.36,2.5,0.0
1,2,2024-01-31 23:59:24,2024-02-01 00:06:13,1,1.28,1,N,68,137,2,9.3,1.0,0.5,0.00,0.0,1.0,14.30,2.5,0.0
2,2,2024-01-31 23:57:33,2024-02-01 00:05:48,1,1.40,1,N,90,79,1,10.0,1.0,0.5,1.95,0.0,1.0,16.95,2.5,0.0


# Metadata Table

We can get more details of an iceberg by looking at its metadata tables. 

## Partitions
For example, to learn about existing partitions in the table, we can query the `partitions` metadata table

In [23]:
nyc_taxis_tbl.inspect.partitions().to_pandas()

,partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
0,"{'tpep_pickup_datetime_year': None, 'tpep_pick...",2,3007511,1,52102669,0,0,0,0,2025-04-03 02:22:37.199,5248436113935406231
1,"{'tpep_pickup_datetime_year': None, 'tpep_pick...",2,11,1,8041,0,0,0,0,2025-04-03 02:22:37.199,5248436113935406231
2,"{'tpep_pickup_datetime_year': None, 'tpep_pick...",2,1,1,7501,0,0,0,0,2025-04-03 02:22:37.199,5248436113935406231
3,"{'tpep_pickup_datetime_year': None, 'tpep_pick...",2,1,1,7501,0,0,0,0,2025-04-03 02:22:37.199,5248436113935406231
4,"{'tpep_pickup_datetime_year': None, 'tpep_pick...",2,2,1,7565,0,0,0,0,2025-04-03 02:22:37.199,5248436113935406231
5,"{'tpep_pickup_datetime_year': 54.0, 'tpep_pick...",1,2964609,1,51732500,0,0,0,0,2025-04-03 02:13:26.142,6445851743841051392
6,"{'tpep_pickup_datetime_year': 53.0, 'tpep_pick...",1,10,1,8028,0,0,0,0,2025-04-03 02:13:26.142,6445851743841051392
7,"{'tpep_pickup_datetime_year': 39.0, 'tpep_pick...",1,3,1,7619,0,0,0,0,2025-04-03 02:13:26.142,6445851743841051392
8,"{'tpep_pickup_datetime_year': 32.0, 'tpep_pick...",1,2,1,7541,0,0,0,0,2025-04-03 02:13:26.142,6445851743841051392


## Files

If we want to see all the data files in the table, we can query the `files` metadata table

In [25]:
nyc_taxis_tbl.inspect.files().to_pandas()

,content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,2,3007511,52102669,"[(1, 351886), (2, 13833395), (3, 14013827), (4...","[(1, 3007511), (2, 3007511), (3, 3007511), (4,...","[(1, 0), (2, 0), (3, 0), (4, 185610), (5, 0), ...",[],"[(1, b'\x01\x00\x00\x00'), (2, b'\x00\xc05\xad...","[(1, b'\x02\x00\x00\x00'), (2, b'\x80\x1b}\x0e...",None,"[4, 17994575, 35919224]",None,NaN,"{'VendorID': {'column_size': 351886, 'value_co..."
1,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,2,11,8041,"[(1, 90), (2, 184), (3, 183), (4, 110), (5, 18...","[(1, 11), (2, 11), (3, 11), (4, 11), (5, 11), ...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'@\xe5_\x91F\x...","[(1, b'\x02\x00\x00\x00'), (2, b'\x802\xda\xac...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
2,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,2,1,7501,"[(1, 90), (2, 110), (3, 110), (4, 110), (5, 11...","[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'@\xb2,\x91__\...","[(1, b'\x02\x00\x00\x00'), (2, b'@\xb2,\x91__\...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
3,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,2,1,7501,"[(1, 90), (2, 110), (3, 110), (4, 110), (5, 11...","[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'@K^\x89`_\x04...","[(1, b'\x02\x00\x00\x00'), (2, b'@K^\x89`_\x04...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
4,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,2,2,7565,"[(1, 90), (2, 118), (3, 118), (4, 110), (5, 11...","[(1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'\xc0\x1a\x8e\...","[(1, b'\x02\x00\x00\x00'), (2, b'@\xbac\x14\x8...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
5,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,1,2964609,51732500,"[(1, 352804), (2, 13713690), (3, 13898307), (4...","[(1, 2964609), (2, 2964609), (3, 2964609), (4,...","[(1, 0), (2, 0), (3, 0), (4, 140162), (5, 0), ...",[],"[(1, b'\x01\x00\x00\x00'), (2, b'\x00 !\x10\xd...","[(1, b'\x06\x00\x00\x00'), (2, b'\xc0(\xae\xb1...",None,"[4, 18218481, 36321783]",None,NaN,"{'VendorID': {'column_size': 352804, 'value_co..."
6,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,1,10,8028,"[(1, 90), (2, 180), (3, 187), (4, 125), (5, 19...","[(1, 10), (2, 10), (3, 10), (4, 10), (5, 10), ...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'@s\n\xc6\xd6\...","[(1, b'\x02\x00\x00\x00'), (2, b'@\xa5.\x0b\xd...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
7,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,1,3,7619,"[(1, 90), (2, 127), (3, 127), (4, 118), (5, 12...","[(1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'@\xdc\xce\xd7...","[(1, b'\x02\x00\x00\x00'), (2, b'\x00\x8c\x83\...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."
8,0,s3://warehouse/demo_ns/nyc_taxis/data/tpep_pic...,PARQUET,1,2,7541,"[(1, 90), (2, 110), (3, 110), (4, 110), (5, 11...","[(1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",[],"[(1, b'\x02\x00\x00\x00'), (2, b'\xc0\xccv% \x...","[(1, b'\x02\x00\x00\x00'), (2, b'\xc0\xccv% \x...",None,[4],None,NaN,"{'VendorID': {'column_size': 90, 'value_count'..."


## Snapshots

If we want to look at snapshots of the table, we can query the `snapshots` metadata table.

Every time when a data change operation happens, Iceberg will form a new snapshot. In this example, we did 2 append and therefore we will have 2 snapshots

In [26]:
nyc_taxis_tbl.inspect.snapshots().to_pandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2025-04-03 02:13:26.142,6445851743841051392,NaN,append,s3://warehouse/demo_ns/nyc_taxis/metadata/snap...,"[(added-files-size, 51755688), (added-data-fil..."
1,2025-04-03 02:22:37.199,5248436113935406231,6.445852e+18,append,s3://warehouse/demo_ns/nyc_taxis/metadata/snap...,"[(added-files-size, 52133277), (added-data-fil..."


There are more metadata tables available, you can find more information here: https://iceberg.apache.org/docs/nightly/spark-queries/#inspecting-tables

# Interoperability with other engines: Spark

Iceberg tables provides engine/platform interoperability. In above example, we use PyIceberg to perform all table operations, we will show that the tables created by PyIceberg can also be consumed by Spark

First, let's set a spark session

In [33]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
  .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1,org.apache.iceberg:iceberg-aws-bundle:1.8.1")
  .config("spark.sql.catalog.demo.type", "rest")
  .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")  
  .config("spark.sql.catalog.demo.uri", "http://rest:8181")
  .config("spark.sql.catalog.demo.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")       
  .config("spark.sql.catalog.demo.warehouse", "s3://warehouse")
  .config("spark.sql.catalog.demo.s3.endpoint", "http://minio:9000")
  .config("spark.sql.catalog.demo.s3.region", "us-east-1")
  .config("spark.sql.catalog.demo.s3.path-style-access", "true")
).getOrCreate()

We can query the nyc_taxis table we just created

In [35]:
spark.sql("SELECT ID, tpep_pickup_datetime, fare_amount FROM demo.demo_ns.nyc_taxis LIMIT 5").show()

+---+--------------------+-----------+
| ID|tpep_pickup_datetime|fare_amount|
+---+--------------------+-----------+
|  2| 2024-02-01 00:04:45|       20.5|
|  2| 2024-02-01 00:56:31|       31.0|
|  2| 2024-02-01 00:07:50|       70.0|
|  1| 2024-02-01 00:01:49|        9.3|
|  1| 2024-02-01 00:37:35|       15.6|
+---+--------------------+-----------+



We can also query the metadata tables of nyc_taxis in spark. For examle, the `snapshots` metadata table

In [40]:
spark.sql("SELECT * FROM demo.demo_ns.nyc_taxis.snapshots").show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-04-03 02:13:...|6445851743841051392|               NULL|   append|s3://warehouse/de...|{added-files-size...|
|2025-04-03 02:22:...|5248436113935406231|6445851743841051392|   append|s3://warehouse/de...|{added-files-size...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+

